In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib notebook
import asyncio
# local
import oxy
import mqtt
from mqtt import run_subscription, logger

In [ ]:
###### setup async process
###### see also http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Asynchronous.html 

from ipykernel.eventloops import register_integration

@register_integration('asyncio')
def loop_asyncio(kernel):
    '''Start a kernel with asyncio event loop support.'''
    loop = asyncio.get_event_loop()

    def kernel_handler():
        loop.call_soon(kernel.do_one_iteration)
        loop.call_later(kernel._poll_interval, kernel_handler)

    loop.call_soon(kernel_handler)
    try:
        if not loop.is_running():
            loop.run_forever()
    finally:
        loop.run_until_complete(loop.shutdown_asyncgens())
        loop.close()

In [ ]:
%gui asyncio

In [ ]:
class FigureData():

    def __init__(self, fig, data=None):
        self.fig = fig
        self.axs = fig.axes
        self.oo = self.get_oo_init(data)
        self.show()

    def show(self):
        self.oo.draw_sO2(self.axs)
        fig.canvas.draw()
        #plt.draw()
        ## hack for non-notebook usage
        #plt.pause(0.01)

    def get_oo_init(self, oo_prev=None):
        oo = oxy.Oxy()
        if oo_prev is not None:
            oo.data = oo_prev.data.median().to_frame().T
        return oo

In [ ]:
## general 
#debug = True
debug = False

if not debug:
    logger.propagate = False

## loop count
#n = 10
## ... infinite
n = None

## graphics
#plt.show()
axs_n = 4
fig, axs = plt.subplots(axs_n, 1,
                        sharex=True,
                        #tight_layout=True,
                        num='demo')
fig.suptitle('From light intensities to Hb oxygen saturation.')

show = FigureData(fig=fig)

## definition of mqtt consumer_action
def consumer_action(buffer,
                    keep=show.oo, # keep will be merged with new oo data
                    show=show):
    # take new data
    oo = oxy.Oxy(buffer, do_eval=False)
    # update evaluation
    keep.eval()
    # update plot from keep
    show.show()
    # data will be joined in mqtt consumer function
    return oo

## asyncio loop
loop = asyncio.get_event_loop()
_ = asyncio.ensure_future(run_subscription(consumer_action=consumer_action, 
                                           produce_n=n, 
                                           consume_n=n,
                                           consume_keep=show.oo))

